# 1. Подготовка данных

In [1]:
#библиотеки
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score,roc_auc_score
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [3]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


Открыт и изучен файл, 
в таблице содержаться данные о 10 тысячах клиентов.

Данные разделены на признаки,
          3 из которых качественных,10 количественных и 1 целевой признак.

In [4]:
#пропуски
data.isnull().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [5]:
#удалениие пропусков 
data = data.dropna(subset = ['Tenure'])

In [6]:
#проверка на наличие пропусков
data.isnull().sum().sum()

0

In [7]:
#проверка на дубликаты 
data.duplicated().sum()

0

In [ ]:
data.info()

In [ ]:
#удаление столбцов
data = data.drop(['Surname'],axis = 1)
data = data.drop(['CustomerId'],axis = 1)
data = data.drop(['RowNumber'],axis = 1)

In [ ]:
data

Данные были изучены на наличие пропусков и дубликатов.
Строки с пропусками в данных о количестве недвижимости у клиента были удалены.
Дубликаты отсутствуют.
Удалены столбцы с данными об индексе,фамилии и уникальном индификаторе клиента.

# 2. Исследование задачи

In [ ]:
#прямое кодирование 
data_ohe = pd.get_dummies(data,drop_first = True)

In [ ]:
#количественное признаки 
features = data_ohe.drop(['Exited'],axis = 1)
#целевой признак
target = data_ohe['Exited']

In [ ]:
#разделение на выборки 
features_t,features_valid,target_t,target_valid = train_test_split(features,target,test_size = 0.2,random_state = 42)
features_train,features_test,target_train,target_test = train_test_split(features_t,target_t,test_size = 0.2,random_state = 42)

Данные разбиты на три выборки :обучающую,тестовую и валидационную в соотношении 60:20:20

In [ ]:
#выбор гиперпараметров
n_estimators_array = [1, 3, 5, 7, 10, 15,20,25,30]
max_depth_array = [1, 3, 5, 7, 10, 15,20,25]
model_1 = RandomForestClassifier(random_state = 42)
grid = GridSearchCV(model_1, param_grid={'n_estimators': n_estimators_array,'max_depth': max_depth_array})
grid.fit(features_train,target_train)
best_n_estimators = grid.best_estimator_.n_estimators
best_max_depth = grid.best_estimator_.max_depth
print(grid.best_score_, best_n_estimators,best_max_depth)

In [ ]:
#модель №1 без учета дисбаланса
model_1 = RandomForestClassifier(random_state = 42,n_estimators=30,max_depth = 7)
model_1.fit(features_train,target_train)
predictions = model_1.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print('Доля правильных ответов:',accuracy)
print('Средняя квадратичная ошибка',mean_squared_error(target_valid, predictions))
print('Коэфициент детерминации',r2_score(target_valid, predictions))
print('Средняя абсалютная ошибка',mean_absolute_error(target_valid, predictions))
print('F1-мера:',f1_score(target_valid, predictions))
probabilities_valid = model_1.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('AUS_ROC:',roc_auc_score(target_valid,probabilities_one_valid))

In [ ]:
#кросс-валидация модели №1
res = cross_val_score(model_1,features_valid,target_valid, cv=6)
print(np.mean(res))

## Вывод по модели № 1 

Обучена модель без учёта дисбаланса.
Выбраны оптимальные гиперпараметры для увеличения доли правильных ответов.
Несмотря на то, что доля правильных ответов составляет более 70 %, при дисбалансе классов данный метрика не является показательной. 
Поэтому были изучены иные метрики, которые показали, что:
      в среднем модель ошибется в 25 случаях из 100 возможных, 
      качество модели ниже среднего, об этом говорит отрицательный коэфициент детерминации, 
      F1-мера, являясь агрегирующей метрикой полноты и точности модели, дает результат, что модель высокого качества,
      и сравнивая F1-меру с AUS_ROC, то можно заметить, что они примерно одинаковы,однако вторая метрика чуть больше на 9%,
      и обе метрики показывает высокое качество классификатора.

# 3. Борьба с дисбалансом

In [ ]:
#взвешивание классов
target.value_counts()

In [ ]:
#уменьшение выборки 
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [ ]:
#применение функции
features,target = downsample(features,target,0.27)

In [ ]:
#проверка веса классов
target.value_counts()

In [ ]:
#разделение на выборки 
features_t,features_valid,target_t,target_valid = train_test_split(features,target,test_size = 0.2,random_state = 42)
features_train,features_test,target_train,target_test = train_test_split(features_t,target_t,test_size = 0.2,random_state = 42)

Проведена проверка классов на дисбаланс, 
результат показал, что класс с положительным значением превышает клас с отрицательным значением примерно в 4 раза, 
применена функция по уменьшению выборки, 
классы сбалансированы, что значит количество значений положительного и отрицательного класса примерно равно

# Модель  № 2

In [ ]:
#выбор гиперпараметров для модели №2
n_estimators_array = [1, 3, 5, 7, 10, 15,20,25,30]
max_depth_array = [1, 3, 5, 7, 10, 15,20,25]
model_2 = RandomForestClassifier(random_state = 42)
grid = GridSearchCV(model_2, param_grid={'n_estimators': n_estimators_array,'max_depth': max_depth_array})
grid.fit(features_train,target_train)
best_n_estimators = grid.best_estimator_.n_estimators
best_max_depth = grid.best_estimator_.max_depth
print(grid.best_score_, best_n_estimators,best_max_depth)

In [ ]:
#модель №2
model1 = RandomForestClassifier(random_state = 42,max_depth = 7,n_estimators=30)
model1.fit(features_train,target_train)
predictions = model1.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print('Доля правильных ответов:',accuracy)
print('Средняя квадратичная ошибка',mean_squared_error(target_valid, predictions))
print('Коэфициент детерминации',r2_score(target_valid, predictions))
print('Средняя абсалютная ошибка',mean_absolute_error(target_valid, predictions))
print('F1-мера:',f1_score(target_valid, predictions))

In [ ]:
#кросс-валидация модели №2
res = cross_val_score(model_2,features_valid,target_valid, cv=17)
print(np.mean(res))

Обучена модель №2 обучена на сбалансированных данных.
Выбраны оптимальные гиперпараметры для увеличения доли правильных ответов.
Были изучены метрики, которые показали, что:
      в среднем модель ошибется в 25 случаях из 100 возможных, 
      качество модели ниже среднего, об этом говорит отрицательный коэфициент детерминации, 
      однако точность и полнота модели в совокупности дают высокий показатель F1-метрики,а именно 0.73
      кросс-валидация оказалась ненамного ниже,чем доля правильных ответов,
      метрику AUC-ROC изучить на данной модели не удалось.

# Модель  № 3

In [ ]:
#выбор значения гиперпараметра max_depth для модели № 3
max_depth_array = [1, 3, 5,6, 7, 10, 15,20,25]
model_3 = DecisionTreeClassifier()
grid = GridSearchCV(model_3, param_grid={'max_depth': max_depth_array})
grid.fit(features_train,target_train)
best_max_depth = grid.best_estimator_.max_depth
print(grid.best_score_, best_max_depth)

In [ ]:
#модель №3
model_3 = DecisionTreeClassifier(random_state = 123,max_depth = 5)
model_3.fit(features_t,target_t)
predictions = model_3.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print('Доля правильных ответов:',accuracy)
print('Средняя квадратичная ошибка',mean_squared_error(target_valid, predictions))
print('Коэфициент детерминации',r2_score(target_valid, predictions))
print('Средняя абсалютная ошибка',mean_absolute_error(target_valid, predictions))
print('F1-мера:',f1_score(target_valid, predictions))
probabilities_valid = model_1.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('AUS_ROC:',roc_auc_score(target_valid,probabilities_one_valid))

In [ ]:
#auc_roc
probabilities_valid = model_3.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid,probabilities_one_valid)
print(auc_roc)

In [ ]:
#кросс-валидация модели №3
res = cross_val_score(model_3,features_valid,target_valid, cv=10)
print(np.mean(res))

Обучена модель №3 обучена на сбалансированных данных.
Выбраны оптимальные гиперпараметры для увеличения доли правильных ответов.
Были изучены метрики, которые показали, что:
      в среднем модель ошибется в 26 случаях из 100 возможных, 
      качество модели ниже среднего, об этом говорит отрицательный коэфициент детерминации, 
      однако точность и полнота модели в совокупности дают высокий показатель F1-метрики,а именно 0.72
      кросс-валидация оказалась ниже,чем доля правильных ответов, что может свидетельствовать о том, что модель ошибется в 32         случаях их 100 возможных,
      высокий показатель площади под ROC-кривой показывает, что модель обучена качественно.
      F1-мера на 13 % ниже, чем площадь под кривой ошибок, однако оба показателя дают информацию о качестве данного                   классификаторе.

# Модель № 4

In [ ]:
#выбор значения гиперпараметра для модели № 4
max_iter_array = [1, 3, 5, 7, 10, 15,20,25,100]
model_4 = LogisticRegression()
grid = GridSearchCV(model_4, param_grid={'max_iter': max_iter_array})
grid.fit(features_train,target_train)
best_max_iter = grid.best_estimator_.max_iter
print(grid.best_score_, best_max_depth)

In [ ]:
#модель №4
model_4 = LogisticRegression(random_state = 42,max_iter=5)
model_4.fit(features_train,target_train)
predictions = model_4.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print('Доля правильных ответов:',accuracy)
print('Средняя квадратичная ошибка',mean_squared_error(target_valid, predictions))
print('Коэфициент детерминации',r2_score(target_valid, predictions))
print('Средняя абсалютная ошибка',mean_absolute_error(target_valid, predictions))
print('F1-мера:',f1_score(target_valid, predictions))
probabilities_valid = model_4.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('AUS_ROC:',roc_auc_score(target_valid,probabilities_one_valid))

In [ ]:
#кросс-валидация модели №4
res = cross_val_score(model_4,features_valid,target_valid, cv=17)
print(np.mean(res))

Обучена модель №4 обучена на сбалансированных данных.
Выбраны оптимальные гиперпараметры для увеличения доли правильных ответов.
Были изучены метрики, которые показали, что:
      в среднем модель ошибется в 47 случаях из 100 возможных, 
      качество модели низкое, об этом говорит самый никий отрицательный коэфициент детерминации из всех изученных моделей, 
      точность и полнота модели в совокупности дают низкий показатель F1-метрики,а именно 0.5,
      кросс-валидация оказалась ниже,чем доля правильных ответов, что может свидетельствовать о том, что модель ошибется в 47         случаях их 100 возможных,
      данная модель имеет все показатели хуже,чем у иных моделей,
      F1-мера на 3.5 % выше, чем площадь под кривой ошибок, однако оба показателя дают информацию о низком качестве данного                   классификаторе

# Вывод по моделям

Для каждой модели были выбраны оптимальные гперпараметры, и подсчитаны ее показатели.

Качество модели №3 показало наилучший результат со следующими показателями :
Средняя квадратичная ошибка показывает, что в средней из 100 случаев 26 будут будут ошибочны, это же утверждение подтверждает и иной показатель, а именно доля правильных ответов, в данной модели он составляет 0.74. 
Отрицательный коэфициент детерминации говорит о том, что качество модели еще есть куда улучшать, однако относительно иных моделей коэфициент данной модели лучше, чем у остальных моделей.  
F1-мера с результатом 0.75 показывает, что полнота и точность модели находяться на достаточно высоком уровне
Применена проверка на кросс-валидации того, чтобы оценить ожидаемый уровень соответствия модели данным, независимым от тех данных, на которых модель тренировалась. На модели № 3 кросс-валидация показала наибольшую долю правильных отевов, равную 0.72

Исследование метрики AUC-ROC показало, что наилучшее качество содержится в модели № 3, данную метрику на модели №1 и №2 посмотреть не удалось


# 4. Тестирование модели

In [ ]:
#проверка модели №3 на тестовой выборке 
predictions = model_3.predict(features_test)
print('Доля правильных ответов:',accuracy_score(target_test, predictions))
print('Средняя квадратичная ошибка',mean_squared_error(target_test, predictions))
print('Коэфициент детерминации',r2_score(target_test, predictions))
print('Средняя абсалютная ошибка',mean_absolute_error(target_test, predictions))
print('F1-мера:',f1_score(target_test, predictions))

На тестовой выборке показатели ненамного выше, чем на обучающей, что говорит о том, что модель слегка переучена

## Проверка модели на адекватность

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train,target_train)
predictions = dummy_clf.predict(features_valid)
print('Доля правильных ответов:',accuracy_score(target_valid, predictions))
print('Средняя квадратичная ошибка',mean_squared_error(target_valid, predictions))
print('Коэфициент детерминации',r2_score(target_valid, predictions))
print('Средняя абсалютная ошибка',mean_absolute_error(target_valid, predictions))
print('F1-мера:',f1_score(target_valid, predictions))

Для проверки модели на адекватность был использован классификатор в качестве простой базовой линии 
для сравнения с реальными классификаторами. 
Проверка показала, что модель № 3 дает 
      большую долю правильных ответов,а так же имеет лучшее качество модели исходя из каэфициента детерминации,и 
      более высокое значение точности и полноты модели, чем используемым для сравнения классификатор, 
тем самым доказывая адекватность модели.

# Общий вывод

Открыт и изучен файл, в таблице содержаться данные о 10 тысячах клиентов.

Данные разделены на признаки, 3 из которых качественных,10 количественных и 1 целевой признак.

Данные были изучены на наличие пропусков и дубликатов.
Строки с пропусками в данных о количестве недвижимости у клиента были удалены.
Дубликаты отсутствуют.
Удалены столбцы с данными об индексе,фамилии и уникальном индификаторе клиента.

Данные разбиты на три выборки :обучающую,тестовую и валидационную в соотношении 60:20:20




<font color="blue">
Модель №1 
</font>


Обучена модель без учёта дисбаланса.
Выбраны оптимальные гиперпараметры для увеличения доли правильных ответов.
Несмотря на то, что доля правильных ответов составляет более 70 %, при дисбалансе классов данный метрика не является показательной. 
Поэтому были изучены иные метрики, которые показали, что:
      в среднем модель ошибется в 25 случаях из 100 возможных, 
      качество модели ниже среднего, об этом говорит отрицательный коэфициент детерминации, 
      F1-мера, являясь агрегирующей метрикой полноты и точности модели, дает результат, что модель высокого качества,
      и сравнивая F1-меру с AUS_ROC, то можно заметить, что они примерно одинаковы,однако вторая метрика чуть больше на 9%,
      и обе метрики показывает высокое качество классификатора.

Проведена проверка классов на дисбаланс, результат показал, что класс с положительным значением превышает клас с отрицательным значением примерно в 4 раза, применена функция по уменьшению выборки, классы сбалансированы, что значит количество значений положительного и отрицательного класса примерно одинаково



<font color="blue">
Модель №2
</font>


Обучена модель №2 обучена на сбалансированных данных.
Выбраны оптимальные гиперпараметры для увеличения доли правильных ответов.
Были изучены метрики, которые показали, что:
      в среднем модель ошибется в 25 случаях из 100 возможных, 
      качество модели ниже среднего, об этом говорит отрицательный коэфициент детерминации, 
      однако точность и полнота модели в совокупности дают высокий показатель F1-метрики,а именно 0.73
      кросс-валидация оказалась ненамного ниже,чем доля правильных ответов,
      метрику AUC-ROC изучить на данной модели не удалось.
 
 
      
<font color="blue">
Модель №3
</font>


Обучена модель №3 обучена на сбалансированных данных.
Выбраны оптимальные гиперпараметры для увеличения доли правильных ответов.
Были изучены метрики, которые показали, что:
      в среднем модель ошибется в 26 случаях из 100 возможных, 
      качество модели ниже среднего, об этом говорит отрицательный коэфициент детерминации, 
      однако точность и полнота модели в совокупности дают высокий показатель F1-метрики,а именно 0.72
      кросс-валидация оказалась ниже,чем доля правильных ответов, что может свидетельствовать о том, что модель ошибется в 32         случаях их 100 возможных,
      высокий показатель площади под ROC-кривой показывает, что модель обучена качественно.
      F1-мера на 13 % ниже, чем площадь под кривой ошибок, однако оба показателя дают информацию о качестве данного                   классификаторе.
     
     
      
<font color="blue">
Модель №4 
</font>


Обучена модель №4 обучена на сбалансированных данных.
Выбраны оптимальные гиперпараметры для увеличения доли правильных ответов.
Были изучены метрики, которые показали, что:
      в среднем модель ошибется в 47 случаях из 100 возможных, 
      качество модели низкое, об этом говорит самый никий отрицательный коэфициент детерминации из всех изученных моделей, 
      точность и полнота модели в совокупности дают низкий показатель F1-метрики,а именно 0.5,
      кросс-валидация оказалась ниже,чем доля правильных ответов, что может свидетельствовать о том, что модель ошибется в 47         случаях их 100 возможных,
      данная модель имеет все показатели хуже,чем у иных моделей,
      F1-мера на 3.5 % выше, чем площадь под кривой ошибок, однако оба показателя дают информацию о низком качестве данного                   классификаторе



Качество модели №3 показало наилучший результат со следующими показателями :
Средняя квадратичная ошибка показывает, что в средней из 100 случаев 26 будут будут ошибочны, это же утверждение подтверждает и иной показатель, а именно доля правильных ответов, в данной модели он составляет 0.74. 
Отрицательный коэфициент детерминации говорит о том, что качество модели еще есть куда улучшать, однако относительно иных моделей коэфициент данной модели лучше, чем у остальных моделей.  
F1-мера с результатом 0.75 показывает, что полнота и точность модели находяться на достаточно высоком уровне
Применена проверка на кросс-валидации того, чтобы оценить ожидаемый уровень соответствия модели данным, независимым от тех данных, на которых модель тренировалась. На модели № 3 кросс-валидация показала наибольшую долю правильных отевов, равную 0.72

Исследование метрики AUC-ROC показало, что в среднем F1-мера чуть ниже, чем AUC-ROC,однако они одинаково дают результат о качетве модели.

На тестовой выборке показатели ненамного выше, чем на обучающей, что говорит о том, что модель слегка переучена

Для проверки модели на адекватность был использован классификатор в качестве простой базовой линии 
для сравнения с реальными классификаторами. 
Проверка показала, что модель № 3 дает 
      большую долю правильных ответов,а так же имеет лучшее качество модели исходя из каэфициента детерминации,и 
      более высокое значение точности и полноты модели, чем используемым для сравнения классификатор, 
тем самым доказывая адекватность модели.